## Master en ingeniería informática: Datos, Cloud y Gestión TI

# Análisis de información no estructurada

## Generador de letras de canciones

#### Realizado por: Arturo Pérez Sánchez y Jacinto Ruiz Díaz

El objetivo de este proyecto es el de implementar un sistema que permita generar de manera automática letras de canciones imitando el estilo de un artista en particular, realizando previamente una lectura de la letra de sus canciones.

El primer paso a realizar es la extracción de las letras de canciones para su posterior análisis. En nuestro caso, para obtener las letras de las canciones hemos optado por realizar <a href="https://es.wikipedia.org/wiki/Web_scraping"> web scrapping </a> de la página <a href="https://www.azlyrics.com/">azlyrics</a> la cual contiene letras de canciones de una amplia variedad de autores.

Para realizar la extracción de las letras utilizaremos la libreria <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup</a> Porque es una de las librerías de python más completas e intuitivas para realizar esta tarea.

Comenzaremos importandola:

In [2]:
import requests
from bs4 import BeautifulSoup

Una vez importada la librería definiremos algunos Hiperparámetros:
<ul>
    <li><strong>group:</strong> Nombre del grupo musical cuyo estilo deseamos imitar.</li>
    <li><strong>albums:</strong> Lista de albumes del grupo que vamos a analizar (Si se deja vacío se leeran todas).</li>
    <li><strong>wait_time:</strong> Tiempo en segundos que transcurrirá entre una petición y otra.</li>
</ul>

In [3]:
group = 'Lady Gaga'
albums = ['Red And Blue', 'The Fame', 'The Fame Monster', 'Born This Way', 'A Very Gaga Holiday', 'Artpop', 'Joanne', 'Chromatica']
wait_time = 5

Lo primero que tenemos que realizar es una conversión del nombre del grupo al nombre utilizado en la página (todo minúsculas y sin espacios)

In [4]:
group = group.lower().replace(" ", "")

Ahora procedemos a estudiar como se estructuran las URLs del sitio web al que vamos a hacer scrapping:

Concretamente, estamos interesados en __dos__ vistas del sitio:

## La lista de canciones separadas por albumes:
URL de esta página se estructura de la forma 
<h2><center>https://www.azlyrics.com/[initial]/[group].html </center></h2>
Donde __[initial]__ corresponde a la primera letra del grupo* y __[group]__ corresponde al nombre del grupo. Se vería de la siguiente forma:

<img src="images/gaga_albums.PNG" width=600>
<br>

## Letra de cada canción:
En este caso, notaremos que la ruta es la misma pero cambiando [initial] por 'lyrics' y añadiendo el nombre de la canción justo después del nombre del grupo. Con el siguiente resultado:

<h2><center>https://www.azlyrics.com/lyrics/[group]/[song].html </center></h2>
<img src="images/gaga_song.PNG" width=600>


*_Si el nombre del grupo comienza por un carácter especial la variable [initial] tomará el valor '19'_

Con esto, ya podemos estrucuturar la ruta de albumes a partir de los hiperparámetros que habiamos establecido

In [5]:
#Inicializamos la variable initial como el primer caracter del grupo o 19 si no es una letra
initial = group[0] if group[0].isalpha() else '19'

#Prefijo del sitio web
prefix = 'https://www.azlyrics.com'
sufix = '.html'

#Concatenamos las variables para formar la URL completa
albumsUrl = prefix + '/' + initial + '/' + group

Con la url de los albumes ya creados, ya podemos utilizar las librerias anteriormente mencionadas para hacer la llamada a la página y obtener la lista de albumes y los titulos de las canciones que lo componen

In [6]:
# Las claves serán los albumes y los valores un array con las canciones. Las canciones a su vez serán un array de tamaño 3
# El primer valor es el titulo, el segundo el enlace y el tercero la letra de la canción
albumsDict = {}

# Hacemos la llamada para obtener la lista de albumes
r = requests.get(albumsUrl + sufix)
soup = BeautifulSoup(r.text, 'lxml')

res = soup.findAll('div', class_=['album', 'listalbum-item'])

# Recorremos la lista de albumes y para cada uno de ellos guardamos en el diccionario
# el titulo de cada canción y la ruta a la letra de dicha canción
currentAlbum = ''
for div in res:
    if(div.get('class')[0] == 'album'):
        # Este div corresponde a un album, por lo que añadimos una nueva entrada al diccionario
        currentAlbum = div.find('b').contents[0].replace('"', '')
        albumsDict[currentAlbum] = []
    else:
        # Este div corresponde a una canción por lo que la añadimos a los valores del album
        albumsDict[currentAlbum].append([div.contents[0].contents[0], 'https://www.azlyrics.com' + div.contents[0]['href'][2:]])

Ahora tenemos un un diccionario con todos los albumes, y para cada album tenemos todas las canciones que lo componen con sus respectivos enlaces a la letra. Por lo que solo nos faltaría llamar a cada uno de estos enlaces para obtener la letra de cada canción, sin embargo, para reducir el número de llamadas al sitio web primero realizaremos una limpieza de los albumes para quedarnos sólo con los albumes que habíamos indicado que nos interesaban

In [8]:
# Limpieza de albumes
# Del diccionario que hemos creado nos quedamos solo con los albumes 
# que aparezcan en el array de albumes que hemos especificado inicialmente o con todos si no se ha especificado
if(albums):
    albumsDict = {a: albumsDict[a] for a in albums if a in albumsDict}

Ahora si que tenemos todo listo para realizar la lectura de las canciones. Cabe destacar que entre cada llamada dejaremos un espacio de tiempo (indicado en la variable 'wait_time') para evitar que el sitio web nos bloquee la dirección IP, para ello haremos uso de la libreria time.

In [9]:
import time

#Creamos un diccionario donde las claves serán las canciones y el valor la letra de dicha canción
lyricsDict = {}

print('leyendo ' + len(albumsDict) + ' albumes')

#Recorremos la lista de albumes y para cada album recorremos la lista de canciones
for album in albumsDict:
    print ('reading album: ', album)
    for song in albumsDict[album]:
        print ('    reading song: ', song[0])
        
        # Esperamos unos segundos antes de realizar la llamada
        time.sleep(wait_time)
        r = requests.get(song[1])
        soup = BeautifulSoup(r.text, 'lxml')
        
        # Como el div que contiene la letra no tiene ninguna clase ni identificador lo obtendremos a partir del div padre
        column = soup.find('div', class_=['col-xs-12 col-lg-8 text-center'])
        
        #El div con la letra de la cancíon siempre estará en 5º lugar, después de los divs del titulo y de las redes sociales
        raw_lyrics = column.findAll('div')[5].text
        lyricsDict[song[0]] = raw_lyrics

reading album:  Red And Blue
    reading song:  Something Crazy
    reading song:  Wish You Were Here
    reading song:  No Floods
    reading song:  Red And Blue
    reading song:  Words
reading album:  The Fame
    reading song:  Just Dance
    reading song:  LoveGame
    reading song:  Paparazzi
    reading song:  Beautiful, Dirty, Rich
    reading song:  Eh, Eh (Nothing Else I Can Say)
    reading song:  Poker Face
    reading song:  The Fame
    reading song:  Money Honey
    reading song:  Again Again
    reading song:  Boys Boys Boys
    reading song:  Brown Eyes
    reading song:  Summerboy
    reading song:  I Like It Rough
    reading song:  Retro Dance Freak
reading album:  The Fame Monster
    reading song:  Bad Romance
    reading song:  Alejandro
    reading song:  Monster
    reading song:  Speechless
    reading song:  Dance In The Dark
    reading song:  Telephone
    reading song:  So Happy I Could Die
    reading song:  Teeth
reading album:  Born This Way
    reading

Por último, tenemos que fusionar los arrays albumsDict y lyricsDict:

In [11]:
# Recorremos los albumes
for album in albumsDict:
    # Recorremos las canciones de cada album
    for song in albumsDict[album]:
        #A cada canción le añadimos la letra correspondiente
        song.append(lyricsDict[j[0]])

Ahora ya tenemos todas las canciones tal y como queríamos por lo que podemos guardarlas en un json para su posterior análisis

In [21]:
import json 

# El json se guardará en la carpeta data
with open('data/' + group + "-lyrics.json", "w") as outfile: 
    json.dump(albumsDict, outfile)